<a href="https://colab.research.google.com/github/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching/blob/main/UAE-with-LFM/LFM_training/celebA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching.git

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torchvision.transforms as T
from torch import Tensor

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchvision.transforms.functional import to_pil_image
from PIL import Image

import os

print(os.getcwd())

%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/models/celebA_model.ipynb
%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/datasets/celebA_dataset.ipynb

%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/models/celebA_flow_model.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [ ]:
# Get data from drive
import zipfile

def unzip_to_data(zip_path, out_dir='./data'):
    os.makedirs(out_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(out_dir)
    print(f"Zip extracted: {out_dir}")

unzip_to_data('/content/drive/MyDrive/CelebA/list_attr_celeba.csv.zip')
unzip_to_data('/content/drive/MyDrive/CelebA/img_align_celeba.zip')
unzip_to_data('/content/drive/MyDrive/CelebA/list_eval_partition.csv.zip')

In [ ]:
# Hyper-Parameters & Settings
batch_size = 500

In [ ]:
# Data
root_dir = './data/img_align_celeba'

train_dataset = CelebADataset(img_dir=root_dir, attr_path='./data/list_attr_celeba.csv', partition_path='./data/list_eval_partition.csv' , mode='train')
val_dataset   = CelebADataset(img_dir=root_dir, attr_path='./data/list_attr_celeba.csv', partition_path='./data/list_eval_partition.csv', mode='val')
test_dataset  = CelebADataset(img_dir=root_dir, attr_path='./data/list_attr_celeba.csv', partition_path='./data/list_eval_partition.csv', mode='test')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

In [ ]:
# Load the UAE model

path = '/content/results/UAE_CelebA'
model = torch.load(path + '.model', weights_only=False, map_location='cpu')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

To_Uniform(
  (encoder): SimpleEncoder(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc): Sequential(
      (0): Linear(in_features=8192, out_features=2048, bias=True)
      (1): SiLU

In [ ]:
# Create the "results" folder
os.makedirs("/content/results", exist_ok=True)

flow_name = './results/UAE_Latent_FM'
flow = Flow().to('cuda' if torch.cuda.is_available() else 'cpu')


optimizer = torch.optim.Adam(flow.parameters(), 0.0003)
loss_fn = nn.MSELoss()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# --- 1. Encode the data ---
def get_encoded_loader(data_loader, encoder, device='cpu'):
    encoder.eval()
    encoded = []

    with torch.no_grad():
        for x in data_loader:
            x = x.to(device)
            z = encoder(x)
            encoded.append(z.cpu())

    all_z = torch.cat(encoded, dim=0)
    if data_loader == train_loader:
      return torch.utils.data.DataLoader(all_z, batch_size=data_loader.batch_size, shuffle=True)
    elif data_loader == val_loader:
      return torch.utils.data.DataLoader(all_z, batch_size=data_loader.batch_size, shuffle=False)

# --- 2. Create a loader with the encoded data ---
encoded_train_loader = get_encoded_loader(train_loader, model.encoder, device=device)
encoded_val_loader = get_encoded_loader(val_loader, model.encoder, device=device)

In [ ]:
# --- 3. Flow Matching training ---
train_cost = []
val_cost = []
best_val_loss = float('inf')

for epoch in range(1000):
    epoch_losses = []

    flow.train()
    for z1 in encoded_train_loader:

        with torch.no_grad():
          z1 = z1.to(device)
        z0 = torch.rand_like(z1)
        t = torch.rand(len(z1), 1, device=device)

        zt = (1 - t) * z0 + t * z1
        dzt = z1 - z0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=zt)
        loss = loss_fn(pred, dzt)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for z1_val in encoded_val_loader:
            z1_val = z1_val.to(device)
            z0_val = torch.rand_like(z1_val)
            t_val = torch.rand(len(z1_val), 1, device=device)

            zt_val = (1 - t_val) * z0_val + t_val * z1_val
            dzt_val = z1_val - z0_val

            pred_val = flow(t=t_val, x_t=zt_val)
            val_loss = loss_fn(pred_val, dzt_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # Save the best model based on validation loss
    if mean_val_loss < best_val_loss:
        print("Saved!")
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/1000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Optionally, save the final epoch model separately
torch.save(flow, flow_name + '_final.model')

In [ ]:
# Save to CSV file
train_losses = train_cost
val_losses = val_cost

np.savetxt("results/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")